# Data preparation for city similarity project (IBM Applied Data Science Capstone)

- We will characterize the city/neighborhood main venues and services by using the foursquare API.

- We will collect other features by web scrapping, such as services available, airports, etc. Wikipedia will be a resource.

- The main feature will use is the cost of living, which will narrow the cities of destination to compare. The site expastitan.com offers such a service.

### Libraries and tools
- Pandas and numpy for data manipulation.
- BeautifulSoup and requests for web scrapping.
- Foursquare API and geocoder. For venues and geolocation.
- Folium for maps.
- Matplotlib for visualization.
- Dotenv for api keys manipulation.
- Scikit-learn for k-means clustering.

# Methods
First, we need to establish the country and city of origin. Second we define the target country, i.e., the destination. With this information we will use a recommender engine to find similarities between the cities.

When a couple of suitable destination cities are found, we will use k-means clustering to group neighborhoods with similar features, using the Foursquare API. Also, the same analysis we will perform in the neighborhood of origin. We will locate the clusters in a Folium map. 

With all this data we will report the destination neighborhoods that are similar to the neighborhood of origin.


In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
import numpy as np

In [3]:
import pandas as pd

In [4]:
import pickle
import os.path

Load the content with `BeautifulSoup`

## Scraping `expatistan` data

`expatistan.com` computes a collaborative international cost of living index, which is useful to compare cities in terms of the daily expenses of life. They offer a worldwide and by region ranking of cities. Also, it is possible to compare two cities directly in terms of several categories, such as food, housing, clothes, transportation, personal care, and entertainment.

We start with the ranking of cities in North America because we are interested in compare US cities against Canada cities.
The final dataframe will contain the North America cost of living index by city.

In [5]:
#: worldwide, 1: europe, 2: 'north-america'
regions = ('', 'europe', 'north-america', 'latin-america', 'asia', 'middle-east', 'africa', 'oceania')

In [6]:
# north-america
reg = 2

In [7]:
# URL for index for region
URL = 'https://www.expatistan.com/cost-of-living/index/'+regions[reg]

In [8]:
# get the page
page = requests.get(URL)

In [9]:
def parse_rank_entry(entry, cities):
    """ Function to parse city row
    """
    clean_entry = entry.strip()[1:-2]
    if len(clean_entry) > 0:
        elem = clean_entry.split(',')
        # in case we fetch 8 elements
        if len(elem) < 9: 
            e = [""]*9
            e[0:3] = elem[0:3]
            e[3] = ""
            e[4:] = elem[3:]
            elem = e
        clean_elems = []
        for i, e in enumerate(elem):
            clean_elems.append(e.strip("\'"))
        # remove text
        clean_elems[6] = clean_elems[6].replace("Cost of Living Index: ", "")
        cities.append(clean_elems)

In [10]:
# flags to detect if we are in the javascript code and array cities
is_script = False
in_cities = False

cities = []
# iterate in lines
for line in page.text.splitlines():
    if "<script" in line:
        is_script = True   
    if "/script>" in line:
        is_script = False
    if "var cities" in line:
        in_cities = True
        # remove text
        line = line.replace("var cities = ", "")
    # check end of array cities
    if in_cities and '}' in line:
        in_cities = False
    # call function to parse entry
    if is_script and in_cities:
        parse_rank_entry(line, cities)        

In [11]:
# create a new dataframe from cities with proper column names
cities_df = pd.DataFrame(cities, columns=["Latitude", "Longitude", "City", "State", "Score", "Population", "Index", "Code", "Country"])

In [12]:
cities_df

,Latitude,Longitude,City,State,Score,Population,Index,Code,Country
0,32.293,-64.782,Hamilton,,0.005474932179887553,2000,293,hamilton-bermuda,BM
1,37.3928,-122.042,Mountain View,California,0.004280022399872152,74066,260,mountain-view-california,US
2,40.7143,-74.006,New York City,,0.004085841541301134,8008278,255,new-york-city,US
3,37.7793,-122.419,San Francisco,California,0.0036448883426488977,808976,244,san-francisco,US
4,40.7114,-74.0647,Jersey City,New Jersey,0.0030973831949348083,247000,231,jersey-city,US
...,...,...,...,...,...,...,...,...,...
63,49.8844,-97.147,Winnipeg,,-0.002054257058711073,632063,138,winnipeg,CA
64,45.5168,-73.6492,Montreal,,-0.0021269846520018732,3268513,137,montreal,CA
65,35.0845,-106.651,Albuquerque,New Mexico,-0.0025748206782716123,487378,131,albuquerque,US
66,35.1495,-90.049,Memphis,Tennessee,-0.002806491271086955,650100,128,memphis,US


In [13]:
cities_df.City[cities_df.State == '']

0            Hamilton
2       New York City
5     Washington D.C.
20            Toronto
21          Vancouver
38             Ottawa
40           Victoria
51            Calgary
54           Edmonton
56            Kelowna
62            Halifax
63           Winnipeg
64           Montreal
67        Quebec City
Name: City, dtype: object

In [14]:
# standarize names of cities, states
cities_df.at[2, 'City'] = 'New York'
cities_df.at[2, 'State'] = 'New York'
cities_df.at[67, 'City'] = 'Quebec'

In [15]:
cities_df

,Latitude,Longitude,City,State,Score,Population,Index,Code,Country
0,32.293,-64.782,Hamilton,,0.005474932179887553,2000,293,hamilton-bermuda,BM
1,37.3928,-122.042,Mountain View,California,0.004280022399872152,74066,260,mountain-view-california,US
2,40.7143,-74.006,New York,New York,0.004085841541301134,8008278,255,new-york-city,US
3,37.7793,-122.419,San Francisco,California,0.0036448883426488977,808976,244,san-francisco,US
4,40.7114,-74.0647,Jersey City,New Jersey,0.0030973831949348083,247000,231,jersey-city,US
...,...,...,...,...,...,...,...,...,...
63,49.8844,-97.147,Winnipeg,,-0.002054257058711073,632063,138,winnipeg,CA
64,45.5168,-73.6492,Montreal,,-0.0021269846520018732,3268513,137,montreal,CA
65,35.0845,-106.651,Albuquerque,New Mexico,-0.0025748206782716123,487378,131,albuquerque,US
66,35.1495,-90.049,Memphis,Tennessee,-0.002806491271086955,650100,128,memphis,US


In [16]:
cities_df.describe()

,Latitude,Longitude,City,State,Score,Population,Index,Code,Country
count,68,68,68,68,68,68,68,68,68
unique,68,68,68,35,48,68,48,68,3
top,39.9523,-123.119,Detroit,,-0.0010224146963587711,390724,153,halifax,US
freq,1,1,1,13,4,1,4,1,56


In [17]:
#cities_df.head().to_clipboard(sep=',')

In [18]:
URL = 'https://www.expatistan.com/cost-of-living/comparison/dallas/montreal'

In [19]:
page = requests.get(URL)

In [20]:
import re

In [21]:
cl_index=[]
for line in page.text.splitlines():
    if "<title>" in line:
        print(line)
        cl_index=re.findall(r'\d+', line)[0]

                <title>Montreal is 14% cheaper than Dallas, Texas. Jul 2020 Cost of Living.</title>


In [22]:
cl_index

'14'

## Scraping `wikipedia.org`

Many websites offers airport data: `openflights.org`, `ourairports.com`, etc. Because we are interested in international airports, it is easy to extract this information from `wikipedia.org`.
The final tables will contain the city with airports in two countries.

In [23]:
# International airports listed in wikipedia
URL = 'https://en.wikipedia.org/wiki/List_of_international_airports_by_country'
page = requests.get(URL)

In [24]:
# load the content in beautiful soup
soup = BeautifulSoup(page.content, 'html.parser')

In [25]:
# look for tables in page
tables = soup.findAll(class_="wikitable sortable")

In [26]:
def parse_table(tables, first_city):
    """ Function to parse a table (ordered by country) given a city up to the end of the table
    """
    data = []
    for table in tables:
        append_it = False
        rows = table.find_all('tr')

        col_name = [name.text.strip() for name in rows[0].find_all('th')]

        for row in rows[1:]:
            cols = row.find_all('td')
            cols = [element.text.strip() for element in cols]
            if cols[0] == first_city:
                data.append(col_name)
                append_it = True
            if append_it:
                data.append([element for element in cols if element])
    return data

In [27]:
#countries = set([country.find(class_='toctext').string for country in soup.findAll(class_="toclevel-{}".format(i)) for i in range(2, 6)])

In [28]:
# scrape list of countries with an international airport
countries = {}
bad_names = ('Passenger Roles (2011-2020)', 'Africa', 'Americas', 'Caribbean', 'Central America', 'North America', 'South America',
             'Asia', 'Central Asia', 'South Asia', 'Southeast Asia', 'Southwest Asia and the Middle East',
             'Europe', 'West Europe', 'Central Europe', 'Southern Europe', 'East Europe', 'Nordic region', 'United Kingdom',
             'Oceania', 'See also', 'References')
for country in soup.findAll(class_="mw-headline"):
    if country.string not in bad_names:
        countries[country.string] = country.string

In [29]:
# set countries to study. If the country does not have an international airport then it does not appear on the
# dictionary and throws a key error
country1 = countries['Canada']

In [30]:
country2 = countries['United States']

In [31]:
# scrape the first city of each country in the table
city1 = soup.find(id=country1.replace(" ", "_")).find_all_next('a')[1].string

In [32]:
city2 = soup.find(id=country2.replace(" ", "_")).find_all_next('a')[1].string

In [33]:
# scrape the airport table for country1
country1_airports = parse_table(tables, city1)

In [34]:
# scrape the airport table for country2
country2_airports = parse_table(tables, city2)

In [35]:
# create new dataframes
country1_airports_df = pd.DataFrame(country1_airports)

In [36]:
c1_airports_df = country1_airports_df.rename(columns=country1_airports_df.iloc[0]).drop(country1_airports_df.index[0]).reset_index(drop=True)

In [37]:
c1_airports_df

,Location,Airport,IATA Code,Passenger Role,2015 Passengers,2014 Passengers
0,Calgary,Calgary International Airport,YYC,Medium,"15,475,759","15,261,108"
1,Edmonton,Edmonton International Airport,YEG,Medium,"7,981,074","8,240,161"
2,Whitehorse,Erik Nielsen Whitehorse International Airport,YXY,None,None,None
3,Gander,Gander International Airport,YQX,None,None,None
4,Halifax (Goffs),Halifax Stanfield International Airport,YHZ,Medium,"10,897,234","3,663,039"
5,Hamilton,John C. Munro Hamilton International Airport,YHM,Non-Hub,"332,378",None
6,Kelowna,Kelowna International Airport,YLK,Small,None,None
7,London,London International Airport,YXU,Non-Hub,None,None
8,Moncton (Dieppe),Greater Moncton International Airport,YQM,Small,"677,159",None
9,Montreal (Dorval),Montreal-Pierre Elliott Trudeau International ...,YUL,Medium,"15,517,382","14,840,067"


In [38]:
# we standarize the city names to avoid complications
c1_airports_df.at[4, 'Location'] = 'Halifax'
c1_airports_df.at[8, 'Location'] = 'Moncton'
c1_airports_df.at[9, 'Location'] = 'Montreal'
c1_airports_df.at[11, 'Location'] = 'Quebec'
c1_airports_df.at[15, 'Location'] = 'Toronto'
c1_airports_df.at[16, 'Location'] = 'Vancouver'

In [39]:
c1_airports_df

,Location,Airport,IATA Code,Passenger Role,2015 Passengers,2014 Passengers
0,Calgary,Calgary International Airport,YYC,Medium,"15,475,759","15,261,108"
1,Edmonton,Edmonton International Airport,YEG,Medium,"7,981,074","8,240,161"
2,Whitehorse,Erik Nielsen Whitehorse International Airport,YXY,None,None,None
3,Gander,Gander International Airport,YQX,None,None,None
4,Halifax,Halifax Stanfield International Airport,YHZ,Medium,"10,897,234","3,663,039"
5,Hamilton,John C. Munro Hamilton International Airport,YHM,Non-Hub,"332,378",None
6,Kelowna,Kelowna International Airport,YLK,Small,None,None
7,London,London International Airport,YXU,Non-Hub,None,None
8,Moncton,Greater Moncton International Airport,YQM,Small,"677,159",None
9,Montreal,Montreal-Pierre Elliott Trudeau International ...,YUL,Medium,"15,517,382","14,840,067"


In [40]:
country2_airports_df = pd.DataFrame(country2_airports)

In [41]:
c2_airports_df = country2_airports_df.rename(columns=country2_airports_df.iloc[0]).drop(country2_airports_df.index[0]).reset_index(drop=True)

In [42]:
c2_airports_df

,Location,Airport,IATA Code,Passenger Role,2018 Passengers
0,Akron,Akron Executive Airport,AKC,Non-Hub/Reliever,No Commercial Service
1,Albany,Albany International Airport,ALB,Small,"2,848,000 [2]"
2,Albuquerque,Albuquerque International Sunport,ABQ,Medium,"5,258,775 [3]"
3,Anchorage,Ted Stevens Anchorage International Airport,ANC,Medium,"5,176,371[4]"
4,Appleton,Appleton International Airport,ATW,Small,"717,757 [5]"
...,...,...,...,...,...
111,"Washington, D.C.",Ronald Reagan Washington National Airport**,DCA,Large,"22,695,582[38]"
112,"Washington, D.C.",Washington Dulles International Airport,IAD,Large,"24,060,709"
113,West Palm Beach,Palm Beach International Airport,PBI,Medium,"6,513,943"
114,Wilkes-Barre/Scranton,Wilkes-Barre/Scranton International Airport,AVP,Small,"508,825[39]"


In [43]:
c1_airports_df.describe()

,Location,Airport,IATA Code,Passenger Role,2015 Passengers,2014 Passengers
count,19,19,19,17,15,13
unique,19,19,19,4,15,13
top,St. John's,Quebec/Jean Lesage International Airport,YHZ,Small,"15,475,759","38,571,961"
freq,1,1,1,7,1,1


In [44]:
# table for report
#c1_airports_df.to_clipboard(sep=',')

## Languages spoken by destination city

`statcan.gc.ca` provides the census data for 2016. From there, we can obtain the languages spoken at home by region and city.

In [45]:
# first we import the csv file with the lines that belongs to a city in the main table.
lines_df = pd.read_csv("lang_city_ines_in_out.csv", index_col=0)

In [46]:
lines_df

,Geo Name,Lines In,Lines Out
Geo Code,,,
10001,St. John's,990460,1485689
10011,Gander,2476147,2971376
12205,Halifax,5447521,5942750
13305,Moncton,8418895,8914124
24421,Quebec,17333017,17828246
24462,Montreal,24266223,24761452
35505,Ottawa,29218513,29713742
35535,Toronto,37637406,38132635
35537,Hamilton,38132635,38627864


In [47]:
lines_df['Lines In']-lines_df['Lines Out']

Geo Code
10001   -495229
10011   -495229
12205   -495229
13305   -495229
24421   -495229
24462   -495229
35505   -495229
35535   -495229
35537   -495229
35555   -495229
46602   -495229
47705   -495229
47725   -495229
48825   -495229
48835   -495229
59915   -495229
59933   -495229
59935   -495229
60990   -495229
dtype: int64

In [48]:
# We have a list with the Id languages to remove from each dataset
langs_remove_df = pd.read_csv('data/lang_id_toremove.csv', index_col=0)

langs_remove_df

""
Id
6
7
9
20
23
...
252
257
258


In [49]:
def extract_languages_by_city(geo_code, line_in, line_out):
    """ Extract the spoken languages by city
        geo_code : code of city
        line_in : number line of chunk
        line_out : next line of chunk
        returns a dataframe with a LangScore
    """
    # name of output file
    fname = "{}.csv".format(geo_code)
    # get the header
    !head -n 1 data/98-400-X2016345_English_CSV_data.csv > {fname}
    # get the chunk of the file and write out
    sed_args = "'{},{}p;{}q'".format(line_in, line_out-1, line_out)
    !sed -n {sed_args} data/98-400-X2016345_English_CSV_data.csv >> {fname}

    # read the generated file
    temp_df = pd.read_csv(fname)

    # we crete a mask with the languages to filte
    mask = np.in1d(temp_df[temp_df.columns[14]].values, np.asarray(langs_remove_df.index))

    # we drop the languages that corresponds to categories and subgroups
    temp_df = temp_df[~mask]
    
    # we repeat the process for the other columns
    mask = np.in1d(temp_df[temp_df.columns[8]].values, np.asarray(langs_remove_df.index))
    temp_df = temp_df[~mask]

    # group by language and ID. keep last three columns
    languages_by_city_df = temp_df.groupby([temp_df.columns[-5],
                                            'DIM: Language spoken at home (263)'])[temp_df.columns[-3:]].sum()

    # rename the columns
    languages_by_city_df.rename(columns={languages_by_city_df.columns[0]: 'TotalLanguageCount',
                                         languages_by_city_df.columns[1]: 'SingleResponse',
                                         languages_by_city_df.columns[2]: 'MultipleResponses'},
                                inplace=True)

    # rename the indices
    languages_by_city_df.index.names = ['LangId', 'Language']

    # sort for popular languages
    languages_by_city_df.sort_values(by=['TotalLanguageCount'], axis=0, ascending=False, inplace=True)

    # second index (Language) to column
    languages_by_city_df.reset_index(level=1, inplace=True)

    # mask to ignore total counts, official and unofficial languages
    mask = ((languages_by_city_df.index == 1) | (languages_by_city_df.index == 2) | (languages_by_city_df.index == 5))

    # max count for a language
    pop_lang_count = languages_by_city_df.TotalLanguageCount[~mask].max()

    # divide the total counts by pop_lang_count and put it in a new column LangScore
    languages_by_city_df['LangScore'] = languages_by_city_df.TotalLanguageCount.apply(lambda row: row/pop_lang_count)
    
    # return the df languages
    return languages_by_city_df#.head(25)

In [50]:
def get_lang_score(fname = 'data/lang_dfs.pickle'):
    """ Process language data and store it in disk and/or memory
    """
    # dict to return
    lang_dfs = {}
    # check if file exists
    if not os.path.isfile(fname):
        geo_code = []
        lines_in = []
        lines_out = []
        print('Processing cities:')
        for index, row in lines_df.iterrows():   
            geo_code = index    
            lines_in = row.values[1]
            lines_out = row.values[2]
            print("Processing {}... {} {} {}".format(row.values[0], geo_code, lines_in, lines_out))
            lang_dfs[geo_code] = extract_languages_by_city(geo_code, lines_in, lines_out)
        print('Done!')
        print('Writing results to disk.')
        with open(fname, 'wb') as file:
            pickle.dump(lang_dfs, file)
        print('Done!')
    else:
        # open file and load dict if file exists
        print('Reading results from disk.')
        with open(fname, 'rb') as file:
            lang_dfs = pickle.load(file)
        print('Done!')
    return lang_dfs

In [51]:
lang_dfs = get_lang_score()

Reading results from disk.
Done!


In [52]:
len(lang_dfs)

19

In [53]:
# check the names in both tables
mask = np.in1d(c1_airports_df.Location, lines_df['Geo Name'].values)
c1_airports_df.Location[~mask]

Series([], Name: Location, dtype: object)

### Missing values from `expatistan` for Canadian cities

In [54]:
mask = (np.in1d(lines_df['Geo Name'].values, cities_df.City) & (True))

In [55]:
# list of cities without score
lines_df[~mask]

,Geo Name,Lines In,Lines Out
Geo Code,,,
10001,St. John's,990460,1485689
10011,Gander,2476147,2971376
13305,Moncton,8418895,8914124
35555,London,42589696,43084925
47705,Regina,55465650,55960879
47725,Saskatoon,57446566,57941795
60990,Whitehorse,84188932,84684161


In [56]:
# convert to float
cities_df.Score = pd.to_numeric(cities_df.Score, downcast="float")
# convert to int
cities_df.Index = pd.to_numeric(cities_df.Index, downcast="integer")

In [57]:
existing_df = cities_df[cities_df.Country == 'CA'][['Score', 'Index']]

In [58]:
existing_df

,Score,Index
20,0.000321,175
21,-0.000028,169
38,-0.000957,154
40,-0.000957,154
51,-0.001422,147
54,-0.001698,143
56,-0.001769,142
62,-0.002054,138
63,-0.002054,138
64,-0.002127,137


In [59]:
score_missing, index_missing = existing_df.mean()[0], int(existing_df.mean()[1])

In [60]:
index_missing

147

In [61]:
expatistan_df = cities_df[['City', 'State', 'Score', 'Index', 'Country']]

In [62]:
expatistan_df

,City,State,Score,Index,Country
0,Hamilton,,0.005475,293,BM
1,Mountain View,California,0.004280,260,US
2,New York,New York,0.004086,255,US
3,San Francisco,California,0.003645,244,US
4,Jersey City,New Jersey,0.003097,231,US
...,...,...,...,...,...
63,Winnipeg,,-0.002054,138,CA
64,Montreal,,-0.002127,137,CA
65,Albuquerque,New Mexico,-0.002575,131,US
66,Memphis,Tennessee,-0.002806,128,US


In [63]:
for city in list(lines_df[~mask]['Geo Name']):
    expatistan_df = expatistan_df.append({'City': city, 'State': '', 'Score': score_missing,
                                          'Index': index_missing, 'Country': 'CA'}, ignore_index=True)

In [64]:
expatistan_df

,City,State,Score,Index,Country
0,Hamilton,,0.005475,293,BM
1,Mountain View,California,0.004280,260,US
2,New York,New York,0.004086,255,US
3,San Francisco,California,0.003645,244,US
4,Jersey City,New Jersey,0.003097,231,US
...,...,...,...,...,...
70,Moncton,,-0.001421,147,CA
71,London,,-0.001421,147,CA
72,Regina,,-0.001421,147,CA
73,Saskatoon,,-0.001421,147,CA


## USA data
### Airports


In [67]:
# international airports, only one airport by city
c2_airports_df.drop_duplicates(subset='Location', inplace=True)
c2_airports_df

,Location,Airport,IATA Code,Passenger Role,2018 Passengers
0,Akron,Akron Executive Airport,AKC,Non-Hub/Reliever,No Commercial Service
1,Albany,Albany International Airport,ALB,Small,"2,848,000 [2]"
2,Albuquerque,Albuquerque International Sunport,ABQ,Medium,"5,258,775 [3]"
3,Anchorage,Ted Stevens Anchorage International Airport,ANC,Medium,"5,176,371[4]"
4,Appleton,Appleton International Airport,ATW,Small,"717,757 [5]"
...,...,...,...,...,...
110,Tampa,Tampa International Airport,TPA,Large,"21,289,390"
111,"Washington, D.C.",Ronald Reagan Washington National Airport**,DCA,Large,"22,695,582[38]"
113,West Palm Beach,Palm Beach International Airport,PBI,Medium,"6,513,943"
114,Wilkes-Barre/Scranton,Wilkes-Barre/Scranton International Airport,AVP,Small,"508,825[39]"


In [68]:
# get airport data
#!wget http://www.nd.gov/gis/apps/Download/?clipping=Full&amp;coord=ND83-SF&amp;format=SHAPE&amp;layers=NDHUB.AIRPORTS
#https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat

In [69]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
opts = Options()
opts.headless = True

In [71]:
def get_state_from_iata():
    """ Scrapes the state from iata codes
    """
    states = []
    information = []
    for index, row in c2_airports_df.iterrows():
        iata_code = row[2]
        print(index, iata_code, end='.-.')
        
        print('Loading driver ', end=' --')
        driver = webdriver.Firefox(options=opts)
        driver.get('https://www.world-airport-codes.com/')
        print('OK', end='**')
        timeout = 60
        input_name = 'input-group-field'
        try:
            element_present = EC.presence_of_element_located((By.CLASS_NAME, input_name))
            WebDriverWait(driver, timeout).until(element_present)
        except TimeoutException:
            print("Timed out waiting for page to load")
        
        search_form = driver.find_element_by_class_name('input-group-field')
        search_form.send_keys(iata_code)
        search_form.submit()

        timeout = 60
        class_name = 'subheader'
        try:
            element_present = EC.presence_of_element_located((By.CLASS_NAME, class_name))
            WebDriverWait(driver, timeout).until(element_present)
        except TimeoutException:
            print("Timed out waiting for page to load")
            
        infos = driver.find_elements_by_class_name('subheader')
        information.append(infos)
        #print(infos, type(infos), len(infos))
        info = infos
        if len(infos) > 1:
            info = infos[0].text
            state = info.split(', ')[1]
        else:
            state = infos
        print(state, end='...')
        states.append(state)        

        driver.quit()
    return states, infos

In [86]:
# get the state for each airport
state_airport_df = []
fname = 'state_airport_df.csv'
if not os.path.isfile(fname):
    states, infos = get_state_from_iata()
    state_airport_df = pd.DataFrame(states, columns=['State'])
    print('write to CSV')
    state_airport_df.to_csv(fname)
else:
    print('Load from CSV')
    state_airport_df = pd.read_csv(fname, index_col=0)

Load from CSV


In [96]:
state_airport_df.reset_index(drop=True)

,State
0,Ohio
1,New York
2,New Mexico
3,Alaska
4,Wisconsin
...,...
104,Florida
105,District of Columbia
106,Florida
107,Pennsylvania


In [262]:
c2_airports_df.reset_index(drop=True, inplace=True)

In [268]:
c2_airports_state_df = pd.concat([c2_airports_df, state_airport_df], axis=1)

In [269]:
c2_airports_state_df.sort_values(by='State', inplace=True)
c2_airports_state_df.reset_index(drop=True, inplace=True)

In [270]:
c2_airports_state_df.at[22, 'Location'] = 'Washington'
c2_airports_state_df.at[41, 'Location'] = 'Kailua'
c2_airports_state_df.at[50, 'Location'] = 'Covington'

In [271]:
c2_airports_state_df

,Location,Airport,IATA Code,Passenger Role,2018 Passengers,State
0,Huntsville (AL),Huntsville International Airport,HSV,Small,TBA,Alabama
1,Birmingham,Birmingham-Shuttlesworth International Airport,BHM,Small,"2,972,776 [8]",Alabama
2,Juneau,Juneau International Airport,JNU,Non-Hub,TBA,Alaska
3,Ketchikan,Ketchikan International Airport,KTN,Non-Hub,TBA,Alaska
4,Fairbanks,Fairbanks International Airport,FAI,Small,"1,078,085[18]",Alaska
...,...,...,...,...,...,...
104,Spokane,Spokane International Airport,GEG,Medium,"3,998,272",Washington
105,Green Bay,Green Bay–Austin Straubel International Airport,GRB,Small,"~650,000[21]",Wisconsin
106,Racine,Batten International Airport,RAC,Small,Unknown (private),Wisconsin
107,Appleton,Appleton International Airport,ATW,Small,"717,757 [5]",Wisconsin


### Transport

In [182]:
# list of states
states_df = pd.read_csv('data/us-states.csv')

In [277]:
states_df.head()

,States
0,Alabama
1,Alaska
2,Arizona
3,Arkansas
4,California


In [202]:
def approx_name(name1, name2):
    if name1 == name2:
        return True
    if name1 in name2:
        return True
    if name2 in name1:
        return True
    return False

In [288]:
# iterate in states
tr_scores = []
for index, row in c2_airports_state_df.iterrows():
        iata_code = row[2]
        city = row[0]
        state = row[5]
        # find index for state to look up in transportation data
        idx = states_df.index[states_df.States == state][0]
        
        if states_df.loc[idx][0]:
            df = pd.read_csv('data/alltransit-{}.csv'.format(idx))
            df.dropna(inplace=True)
            mask = []
            for index2, row2 in df.iterrows():
                #print(acity)
                try:
                    acity = str(row2[1]).strip('\"')
                except IndexError as e:
                    #print('no rows ', idx, state, acity, city, type(acity))
                    pass
                mask.append(approx_name(acity, city))
            if np.all(np.asarray(mask) == False):
                print(idx, state, city, acity, type(acity))
                print('missed')
            else:
                idx2 = df.index[mask][0]           
                tr_scores.append(df.loc[idx2][['blkgrps','population', 'households', 'alltransit_performance_score']])

In [289]:
tr_scores = pd.DataFrame(tr_scores, )

In [293]:
transport_scores_df = pd.concat([c2_airports_state_df.reset_index(drop=True), tr_scores.reset_index(drop=True)], axis=1)

In [297]:
us_transport_scores_df = transport_scores_df.copy()
us_transport_scores_df.head()

,Location,Airport,IATA Code,Passenger Role,2018 Passengers,State,blkgrps,population,households,alltransit_performance_score
0,Huntsville (AL),Huntsville International Airport,HSV,Small,TBA,Alabama,126.22,186049.43,78580.94,1.8
1,Birmingham,Birmingham-Shuttlesworth International Airport,BHM,Small,"2,972,776 [8]",Alabama,213.55,216700.33,91271.18,0.2
2,Juneau,Juneau International Airport,JNU,Non-Hub,TBA,Alaska,23.00,32434.00,12273.00,2.6
3,Ketchikan,Ketchikan International Airport,KTN,Non-Hub,TBA,Alaska,4.85,5467.42,2215.36,4.7
4,Fairbanks,Fairbanks International Airport,FAI,Small,"1,078,085[18]",Alaska,17.98,22293.81,8806.83,4.3


In [295]:
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#    print(c2_airports_state_df)

#### Canada transport score

In [296]:
!wget -O transport_canada_scores.csv https://www150.statcan.gc.ca/n1/daily-quotidien/200602/t001a-eng.csv

--2020-07-17 09:47:05--  https://www150.statcan.gc.ca/n1/daily-quotidien/200602/t001a-eng.csv
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving www150.statcan.gc.ca (www150.statcan.gc.ca)... 205.193.226.160
Connecting to www150.statcan.gc.ca (www150.statcan.gc.ca)|205.193.226.160|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8385 (8.2K) [text/csv]
Saving to: ‘transport_canada_scores.csv’

transport_canada_sc 100%[===================>]   8.19K  --.-KB/s    in 0.03s   

2020-07-17 09:47:05 (316 KB/s) - ‘transport_canada_scores.csv’ saved [8385/8385]



In [360]:
transport_scores_df = pd.read_csv('transport_canada_scores.csv', skiprows=[0,1,2], names=["City", "Total", "500m", "Score"])

In [361]:
transport_scores_df.head()

,City,Total,500m,Score
0,"Toronto, Ontario","5,928",...,93.0
1,"Vancouver, British Columbia","2,463",...,92.7
2,"Montréal, Quebec","4,099",...,91.6
3,"Regina, Saskatchewan",236,...,90.4
4,"Victoria, British Columbia",368,...,90.4


In [362]:
# we need to split city and province
city_province = transport_scores_df['City'].str.split(", ", n=1, expand=True)

In [363]:
transport_scores_df['City'] = city_province[0]
transport_scores_df['Province'] = city_province[1]

In [364]:
# drop irrelevant last rows
transport_scores_df.drop(transport_scores_df.tail(5).index, inplace=True)

In [365]:
# utility to remove accents in strings
import unicodedata

def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return u"".join([c for c in nfkd_form if not unicodedata.combining(c)])

In [366]:
# remove accents in city names
transport_scores_df['City'] = transport_scores_df['City'].apply(remove_accents)

In [367]:
transport_scores_df.head()

,City,Total,500m,Score,Province
0,Toronto,"5,928",...,93.0,Ontario
1,Vancouver,"2,463",...,92.7,British Columbia
2,Montreal,"4,099",...,91.6,Quebec
3,Regina,236,...,90.4,Saskatchewan
4,Victoria,368,...,90.4,British Columbia


In [368]:
# check the names in both tables
mask = np.in1d(c1_airports_df.Location, transport_scores_df['City'].values)
c1_airports_df.Location[~mask]

2    Whitehorse
3        Gander
Name: Location, dtype: object

In [369]:
# convert to float
transport_scores_df.Score = pd.to_numeric(transport_scores_df.Score, downcast="float")

In [370]:
# get the mean score
mean_score = transport_scores_df.Score.mean()

In [371]:
transport_scores_df.loc[len(transport_scores_df)] = ['Whitehorse', 0, 0, mean_score, 'Yukon']
transport_scores_df.loc[len(transport_scores_df)] = ['Gander', 0, 0, mean_score, 'Newfoundland and Labrador']

In [372]:
# check the names in both tables
mask = np.in1d(c1_airports_df.Location, transport_scores_df['City'].values)
c1_airports_df.Location[~mask]

Series([], Name: Location, dtype: object)

In [373]:
mask = np.in1d(transport_scores_df['City'].values, c1_airports_df.Location)

In [375]:
canada_transport_scores_df = transport_scores_df[mask].reset_index(drop=True)

In [376]:
canada_transport_scores_df

,City,Total,500m,Score,Province
0,Toronto,"5,928",...,93.000000,Ontario
1,Vancouver,"2,463",...,92.699997,British Columbia
2,Montreal,"4,099",...,91.599998,Quebec
3,Regina,236,...,90.400002,Saskatchewan
4,Victoria,368,...,90.400002,British Columbia
5,Calgary,"1,393",...,88.900002,Alberta
6,Winnipeg,778,...,88.199997,Manitoba
7,Ottawa,992,...,85.000000,Ontario
8,Quebec,800,...,83.099998,Quebec
9,Edmonton,"1,321",...,82.699997,Alberta


### Canada parks/green area scores

In [378]:
# URL for index for region
URL = 'https://www150.statcan.gc.ca/n1/pub/16-201-x/2016000/tbl/tbl2-4-eng.htm'

In [379]:
# get the page
page = requests.get(URL)

In [614]:
#page.text

In [385]:
# load the content in beautiful soup
soup = BeautifulSoup(page.content, 'html.parser')

In [388]:
# look for table in page
table = soup.find(class_="pub-table")

In [496]:
def parse_table_parks(table):
    """ Parse the table of parks access
    """
    data = []
    rows = table.find_all(class_="highlight-row")

    for row in rows[1:]:
        city = row.find_all(class_='stub-indent1')[0].string
        if city == 'Single-detached':
            break
        if city is not None:
            cols = [float(r.contents[0]) for r in row.find_all('td')]
            data.append([remove_accents(city.replace(u'\xa0', u' ')), *cols])
            
    return data

In [498]:
canada_parks = parse_table_parks(table)

In [501]:
canada_parks_df = pd.DataFrame(canada_parks, columns=['City', 'Parks_Score', 'Outdoor_Score', 'Trees_Score', 'Plant_Score', 'Birds_Score'])

In [503]:
canada_parks_df.head()

,City,Parks_Score,Outdoor_Score,Trees_Score,Plant_Score,Birds_Score
0,Toronto,85.0,69.0,78.0,52.0,19.0
1,Montreal,91.0,66.0,83.0,48.0,17.0
2,Edmonton,89.0,80.0,84.0,63.0,28.0
3,Vancouver,87.0,75.0,74.0,53.0,19.0
4,Calgary,88.0,69.0,81.0,54.0,23.0


### USA parks/green area scores

In [747]:
URL = 'https://www.tpl.org/parkscore'
page = requests.get(URL)

In [748]:
driver = webdriver.Firefox()
driver.get(URL)

In [749]:
driver.execute_script("window.scrollTo(0, 1500)")

In [750]:
select_box = driver.find_element_by_xpath("//input[@placeholder='See All Parkscore Rankings']")

In [751]:
select_box.click()

In [752]:
html = driver.page_source
park_soup = BeautifulSoup(html, "lxml")

In [755]:
driver.quit()

In [753]:
autocompletes = park_soup.find_all(class_='ui-autocomplete')

In [782]:
#[au.string.split(' ', 1)[1].split(',')[0] for au in autocompletes[2].find_all(class_='ui-corner-all')]

In [756]:
us_cities = [au.string.split(' ', 1)[1].split(',')[0] for au in autocompletes[2].find_all(class_='ui-corner-all')]

In [772]:
# get the cities with park index
mask = np.in1d(c2_airports_state_df.Location, us_cities)
us_cities_in = c2_airports_state_df[mask][['Location', 'State']]

In [773]:
us_cities_in = (us_cities_in.Location+'-'+us_cities_in.State).str.lower()

In [775]:
us_cities_in = us_cities_in.str.replace(' ','-')

In [777]:
us_cities_in = us_cities_in.str.replace('\.-','-')

In [783]:
us_cities_in = us_cities_in.str.replace('-of','')

In [865]:
usa_parks = []
for city, City in zip(list(us_cities_in), list(c2_airports_state_df[mask]['Location'])):
    URL = 'https://www.tpl.org/city/'+city
    print(URL, end=" ...")
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    #if 'Page not found' in soup.text:
    #    print(city)


    soup.find_all(class_="city-name")
    rank = soup.find(class_='city-rank')
    #print(rank)
    if rank is not None:
        try:
            rank = float(rank.find("strong").string.strip('\#'))
        except AttributeError as e:
            rank = None
            pass
#     else:
#         print('URL')

    access_rank = soup.find(class_="parkscore-characteristics-graph-blurb")
    #print(access_rank)
    if access_rank is not None:
        try:
            access_rank = float(access_rank.find("strong").string)
        except AttributeError as e:
            access_rank = None

    usa_parks.append([City, rank, access_rank])

https://www.tpl.org/city/anchorage-alaska ...https://www.tpl.org/city/phoenix-arizona ...https://www.tpl.org/city/fresno-california ...https://www.tpl.org/city/los-angeles-california ...https://www.tpl.org/city/san-francisco-california ...https://www.tpl.org/city/oakland-california ...https://www.tpl.org/city/sacramento-california ...https://www.tpl.org/city/san-diego-california ...https://www.tpl.org/city/denver-colorado ...https://www.tpl.org/city/washington-district-columbia ...https://www.tpl.org/city/miami-florida ...https://www.tpl.org/city/jacksonville-florida ...https://www.tpl.org/city/orlando-florida ...https://www.tpl.org/city/tampa-florida ...https://www.tpl.org/city/atlanta-georgia ...https://www.tpl.org/city/honolulu-hawaii ...https://www.tpl.org/city/boise-idaho ...https://www.tpl.org/city/chicago-illinois ...https://www.tpl.org/city/indianapolis-indiana ...https://www.tpl.org/city/louisville-kentucky ...https://www.tpl.org/city/new-orleans-louisiana ...https://www.tpl.o

In [998]:
usa_parks_df = pd.DataFrame(usa_parks, columns=["city", "rank", "access_rank"])

In [999]:
mean_rank, mean_arank = usa_parks_df.dropna().mean()

In [1000]:
idx = usa_parks_df[usa_parks_df['rank'].isnull()]['rank'].index[0]
usa_parks_df.loc[idx, 'rank'] = mean_rank
idx = usa_parks_df[usa_parks_df['access_rank'].isnull()]['access_rank'].index[0]
usa_parks_df.loc[idx, 'access_rank'] = mean_arank

In [1001]:
usa_parks_df.head()

,city,rank,access_rank
0,Anchorage,39.0,62.0
1,Phoenix,56.0,22.0
2,Fresno,92.0,51.0
3,Los Angeles,49.0,51.0
4,San Francisco,8.0,100.0


In [1002]:
mask2 = np.in1d(c2_airports_state_df.Location, usa_parks_df)

In [1013]:
usa_parks_df = pd.concat([c2_airports_state_df[mask2][['Location', 'State']].reset_index(drop=True), usa_parks_df[['rank', 'access_rank']].reset_index(drop=True)], axis=1)

In [1014]:
usa_parks_out = c2_airports_state_df[~mask2][['Location', 'State']]

In [1015]:
usa_parks_out['rank'] = mean_rank

In [1016]:
usa_parks_out['access_rank'] = access_rank

In [1017]:
usa_parks_out.head()

,Location,State,rank,access_rank
0,Huntsville (AL),Alabama,44.931818,86.0
1,Birmingham,Alabama,44.931818,86.0
2,Juneau,Alaska,44.931818,86.0
3,Ketchikan,Alaska,44.931818,86.0
4,Fairbanks,Alaska,44.931818,86.0


In [1018]:
us_parks_ranks_df = pd.concat([usa_parks_df.reset_index(drop=True), usa_parks_out.reset_index(drop=True)], axis=0)

In [1019]:
us_parks_ranks_df

,Location,State,rank,access_rank
0,Anchorage,Alaska,39.000000,62.0
1,Phoenix,Arizona,56.000000,22.0
2,Fresno,California,92.000000,51.0
3,Los Angeles,California,49.000000,51.0
4,San Francisco,California,8.000000,100.0
...,...,...,...,...
59,Snohomish County (WA),Washington,44.931818,86.0
60,Spokane,Washington,44.931818,86.0
61,Green Bay,Wisconsin,44.931818,86.0
62,Racine,Wisconsin,44.931818,86.0


### expatistan scores

In [1146]:
usa_expatistan = expatistan_df[expatistan_df['Country']=='US'][['City', 'Index', 'Score']]

In [1147]:
usa_expatistan.reset_index(drop=True, inplace=True)
usa_expatistan.sort_values(by='City', inplace=True)

In [1148]:
usa_expatistan_df = c2_airports_state_df.copy().sort_values(by='Location')

In [1149]:
len(usa_expatistan)

56

In [1153]:
mask = np.in1d(usa_expatistan_df['Location'], usa_expatistan['City'])
mask2 = np.in1d(usa_expatistan['City'], usa_expatistan_df['Location'])

In [1154]:
len(usa_expatistan_df[mask])

41

In [1155]:
len(usa_expatistan_df)

109

In [1156]:
len(usa_expatistan[mask2])

41

In [1173]:
usa_expatistan_score_in = pd.concat([usa_expatistan_df[mask][['Location', 'State']].reset_index(drop=True), usa_expatistan[mask2][['Index', 'Score']].reset_index(drop=True)], axis=1).reset_index(drop=True)

In [1174]:
usa_expatistan_score_in.head()

,Location,State,Index,Score
0,Albuquerque,New Mexico,131,-0.002575
1,Anchorage,Alaska,182,0.000713
2,Atlanta,Georgia,164,-0.000328
3,Austin,Texas,167,-0.000147
4,Baltimore,Maryland,161,-0.000513


In [1175]:
index_mean, score_mean = usa_expatistan.mean()

In [1176]:
usa_expatistan_out = c2_airports_state_df[~mask][['Location', 'State']]

In [1177]:
usa_expatistan_out['Index'] = index_mean

In [1178]:
usa_expatistan_out['Score'] = score_mean

In [1182]:
usa_expatistan_score_df = pd.concat([usa_expatistan_score_in, usa_expatistan_out])

In [1183]:
usa_expatistan_score_df.head()

,Location,State,Index,Score
0,Albuquerque,New Mexico,131.0,-0.002575
1,Anchorage,Alaska,182.0,0.000713
2,Atlanta,Georgia,164.0,-0.000328
3,Austin,Texas,167.0,-0.000147
4,Baltimore,Maryland,161.0,-0.000513


In [1186]:
canada_expatistan = expatistan_df[expatistan_df['Country']=='CA'][['City', 'Index', 'Score']]

In [1187]:
canada_expatistan.reset_index(drop=True, inplace=True)
canada_expatistan.sort_values(by='City', inplace=True)

In [1189]:
canada_expatistan_df = c1_airports_df.copy().sort_values(by='Location')

In [1190]:
mask = np.in1d(canada_expatistan_df['Location'], canada_expatistan['City'])
mask2 = np.in1d(canada_expatistan['City'], canada_expatistan_df['Location'])

In [1212]:
canada_expatistan_score_in = pd.concat([canada_expatistan_df[mask][['Location']].reset_index(drop=True), canada_expatistan[mask2][['Index', 'Score']].reset_index(drop=True)], axis=1).reset_index(drop=True)

In [1213]:
index_mean, score_mean = canada_expatistan.mean()

In [1214]:
canada_expatistan_out = c1_airports_df[~mask][['Location']]

In [1215]:
canada_expatistan_out['Index'] = index_mean

In [1216]:
canada_expatistan_out['Score'] = score_mean

In [1217]:
canada_expatistan_score_df = pd.concat([canada_expatistan_score_in, canada_expatistan_out]).reset_index(drop=True)

In [1220]:
canada_expatistan_score_df

,Location,Index,Score
0,Calgary,147.000000,-0.001422
1,Edmonton,143.000000,-0.001698
2,Gander,147.000000,-0.001421
3,Halifax,138.000000,-0.002054
4,Kelowna,142.000000,-0.001769
5,London,147.000000,-0.001421
6,Moncton,147.000000,-0.001421
7,Montreal,137.000000,-0.002127
8,Ottawa,154.000000,-0.000957
9,Quebec,127.000000,-0.002885


In [1221]:
canada_transport_scores_df,us_transport_scores_df, canada_parks_df,us_parks_ranks_df, usa_expatistan_score_df, canada_expatistan_score_df

(          City                                    Total  \
 0      Toronto                                    5,928   
 1    Vancouver                                    2,463   
 2     Montreal                                    4,099   
 3       Regina                                      236   
 4     Victoria                                      368   
 5      Calgary                                    1,393   
 6     Winnipeg                                      778   
 7       Ottawa                                      992   
 8       Quebec                                      800   
 9     Edmonton                                    1,321   
 10   Saskatoon                                      295   
 11    Hamilton                                      748   
 12      London                                      494   
 13     Kelowna                                      195   
 14     Halifax                                      403   
 15     Moncton                         

## Neighborhood segmentation for origin and destination cities

This is a mock test, because we need to do the recommendation analysis before doing the clustering.

Let's assume that the destination city is Montreal (CA), and the city of origin is Akron (USA).

Foursquare API
 * 99 regular calls per day
 
 1. Regular: Search for a specific type of venue around a given location.
 
 2. Premium: learn more about a specific venue
 
 3. Regular: learn about a user.
 
 4. Regular: explore a given location.
 
 5. Regular: explore trending venues around a given location.

http://api.foursquare.com/v2/tips

venues

users 

tips

In [30]:
# load environment variables
import os
from dotenv import load_dotenv
load_dotenv()

True

In [31]:
# get foursquare key
CLIENT_ID = os.getenv('FOURSQUARE_CLIENT_ID')# your Foursquare ID
CLIENT_SECRET = os.getenv('FOURSQUARE_CLIENT_SECRET')# your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [32]:
# get bing api key
bing_api_key = os.environ.get('BING_API_KEY')

In [33]:
c1_airports_df.Location[9].strip()

'Montreal (Dorval)'

In [34]:
# regular expressions
import re

In [35]:
def clean_city(city):
    """ Remove parenthesis and text between from city
    """
    city = re.sub("[\(\[].*?[\)\]]", "", city)
    return city.replace(" ", "") 

In [36]:
# apply function to each city
c1_airports_df.Location = c1_airports_df.Location.apply(clean_city)

In [37]:
c1_airports_df = c1_airports_df.rename(columns={'Location': 'City'})

In [38]:
dest_city = c1_airports_df.City[9]

In [39]:
print(dest_city)

Montreal


In [40]:
# find postal codes
URL = 'https://worldpostalcode.com/canada/quebec/' + dest_city.lower()
page = requests.get(URL)

Load the content with `BeautifulSoup`

In [41]:
soup = BeautifulSoup(page.content, 'html.parser')

The table to scrape is identified by `class="codes"`

In [42]:
table = soup.find(class_="codes")

Parse the table

In [44]:
places = table.find_all(class_ = 'place')
codes = table.find_all(class_ = 'code')
#print(places)
data = []
for place, code in zip(places, codes):
    data.append((place.string, code.string))

Load the table into a dataframe

In [45]:
data

[('Beaconsfield', 'H9W'),
 ('Cote-Saint-Luc West', 'H4W'),
 ('Dorval Outskirts', 'H9P'),
 ('Downtown Montreal East', 'H3B'),
 ('Downtown Montreal North', 'H3A'),
 ('Downtown Montreal Northeast', 'H2Z'),
 ('Downtown Montreal South & West', 'H3H'),
 ('Downtown Montreal Southeast', 'H3G'),
 ('Hampstead', 'H3X'),
 ('Kirkland', 'H9J'),
 ('Montreal East', 'H1B'),
 ('Montreal North North', 'H1G'),
 ('Montreal North South', 'H1H'),
 ('Montreal West', 'H4X'),
 ('Old Montreal', 'H2Y'),
 ('Pointe-Claire', 'H9R'),
 ('Westmount East', 'H3Z'),
 ('Westmount West', 'H3Y')]

In [46]:
data = pd.DataFrame(data, columns=['Borough', 'PostalCode'])

In [47]:
data

,Borough,PostalCode
0,Beaconsfield,H9W
1,Cote-Saint-Luc West,H4W
2,Dorval Outskirts,H9P
3,Downtown Montreal East,H3B
4,Downtown Montreal North,H3A
5,Downtown Montreal Northeast,H2Z
6,Downtown Montreal South & West,H3H
7,Downtown Montreal Southeast,H3G
8,Hampstead,H3X
9,Kirkland,H9J


Check for repetitions for `PostalCode`

### Geolocation data by bing!

In [48]:
import geocoder

Use geocoder and bing to obtain the coordinates for each postal code

In [49]:
coords = []
for i in data.index:
    postal_code = data.at[i, 'PostalCode']

    lat_lng_coords = None

    g = []
    while(not g):
        g = geocoder.bing(location=dest_city, postalCode='{}'.format(postal_code), method='details', key=bing_api_key)
        
    
    lat_lng_coords = g.latlng

    coords.append([lat_lng_coords[0], lat_lng_coords[1]])

In [50]:
dest_city_data = pd.concat([data, pd.DataFrame(coords)], axis=1).sort_values(by='PostalCode').reset_index(drop=True)

In [51]:
dest_city_data.rename(columns={0: 'Latitude', 1: 'Longitude'}, inplace=True)

Table with coordinates retrieved using bing

In [85]:
dest_city_data.to_clipboard(sep=',')

In [54]:
import folium
import json

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium # map rendering library

In [56]:
address = dest_city+','+country1

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))

The geograpical coordinate of Montreal,Canada are 45.4972159, -73.6103642.


In [58]:

map_dest_city = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough in zip(dest_city_data['Latitude'], 
                             dest_city_data['Longitude'], 
                             dest_city_data['Borough']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_dest_city)  
    
map_dest_city

In [87]:
# to save a PNG of the map
import io
from PIL import Image

In [88]:
img_data = map_dest_city._to_png(5)
img = Image.open(io.BytesIO(img_data))
img.save('map_dest_city.png')

![image](map_dest_city.png)

In [59]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### Explore Neighborhoods

#### Use the same function of previous Lab

In [60]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Run the function

In [62]:
LIMIT = 100
radius = 500
dest_city_venues = getNearbyVenues(names=dest_city_data['Borough'],
                                   latitudes=dest_city_data['Latitude'],
                                   longitudes=dest_city_data['Longitude']
                                  )



Montreal East
Montreal North North
Montreal North South
Old Montreal
Downtown Montreal Northeast
Downtown Montreal North
Downtown Montreal East
Downtown Montreal Southeast
Downtown Montreal South & West
Hampstead
Westmount West
Westmount East
Cote-Saint-Luc West
Montreal West
Kirkland
Dorval Outskirts
Pointe-Claire
Beaconsfield


#### Let's check the results

In [63]:
print(dest_city_venues.shape)
dest_city_venues.head()

(492, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Montreal East,45.62912,-73.50468,Broadway Pizzeria,45.627372,-73.506175,Restaurant
1,Montreal East,45.62912,-73.50468,Centre Recreatif Edouard-Rivet,45.628681,-73.500580,Recreation Center
2,Montreal East,45.62912,-73.50468,Restaurant Sunshine,45.631091,-73.508728,Sushi Restaurant
3,Montreal East,45.62912,-73.50468,Deli Diane,45.626068,-73.502196,Deli / Bodega
4,Montreal East,45.62912,-73.50468,Couche-Tard,45.626031,-73.502286,Convenience Store


Let's check how many venues were returned for each neighborhood

In [65]:
dest_city_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Beaconsfield,1,1,1,1,1,1
Cote-Saint-Luc West,4,4,4,4,4,4
Dorval Outskirts,2,2,2,2,2,2
Downtown Montreal East,82,82,82,82,82,82
Downtown Montreal North,57,57,57,57,57,57
Downtown Montreal Northeast,100,100,100,100,100,100
Downtown Montreal South & West,9,9,9,9,9,9
Downtown Montreal Southeast,100,100,100,100,100,100
Hampstead,3,3,3,3,3,3


#### Let's find out how many unique categories can be curated from all the returned venues

In [66]:
print('There are {} uniques categories.'.format(len(dest_city_venues['Venue Category'].unique())))

There are 152 uniques categories.


# Data description and methods

From the tables above, we will extract the following features:

- We will focus in the destination cities which have a nearby airport.
- With the origin address and country destination we will determine the most appropiated cities to migrate.
- We will use postal code information to get the borough of the destination cities.
- We will use bing to get the coordinates of the boroughs.
- We will use follium to display the map and the boroughs.
- We will use foursquare API to get the most popular venues in destination and origin cities.
- We will use k-means clustering to cluster the boroughs with common features.
- We will determine the similarity between the borough of origin and each cluster in the city of destination, usign a recommendation engine.